In [9]:
from opentrons import protocol_api

def quick_mix(pipette, well, volume, rep=3,  rate = 8.0, adj_height = 6.0, blow_out = True):
    """ Mix the solution up and down specified number of times
    rep = number of repetitions
    rate = the rate to aspirate and dispense while mixing
    adj_height = how far above the well bottom to dispense the fluid
    """
    old_height = pipette.well_bottom_clearance.dispense
    pipette.well_bottom_clearance.dispense = adj_height
    if not pipette.hw_pipette['has_tip']:
        pipette.pick_up_tip()
    for i in range(rep):
        pipette.aspirate(20, well, rate=rate)
        pipette.dispense(20, well, rate=rate)
 
    if blow_out:
        pipette.blow_out()
        
    pipette.touch_tip()
    pipette.well_bottom_clearance.dispense = old_height
    pipette.drop_tip()

metadata = {'apiLevel': '2.4',
           'protocolName':'Sphingosine Adsorption',
           'author':'Brae',
           'description':'Sphingosine adsorption study'}


def run(protocol:protocol_api.ProtocolContext):
    
    """
    Epimark Nucleosome Assembly using the Dilution method for 50 pmol using their control DNA as a template. 
    """
    tip_20 = protocol.load_labware('opentrons_96_filtertiprack_20ul',1)
    tip_200_locations = [2,5]
    tips_200 = [protocol.load_labware('opentrons_96_filtertiprack_200ul', x) for x in tip_200_locations]

    plates = protocol.load_labware('corning_96_wellplate_360ul_flat',3)
    reservoir = protocol.load_labware('nest_12_reservoir_15ml', 4)
    
    
    p20 = protocol.load_instrument('p20_single_gen2', 'left', tip_racks = [tip_20])
    p300_multi = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks = tips_200)
    

    samples = plates.wells()[0:96]
    pbs_buffer = reservoir.wells()[0]
    h20 = reservoir.wells()[1]
    buffer = reservoir.wells()[2]
    waste = reservoir.wells()[3]
    
    # Step 1-12
    p300_multi.transfer(200, samples, waste, blow_out=True)

    # Step 13-24
    p300_multi.transfer(150, pbs_buffer, samples)
    
    # Step 25-36
    p300_multi.transfer(200, samples, waste, blow_out=True)
    
    # Step 37-48
    p300_multi.transfer(150, h20, samples)
    
    # Steps 49-60
    p300_multi.transfer(200, samples, waste)
    
    # Steps 61-72 
    message = "Add sphingo lipids manually and incubate for 20, press resume to continue"
    
    # Steps 73 - 84
    p300_multi.transfer(300, samples, waste)
    
    # Steps 85-156 Rinse with buffer
    for i in range(3):
        p300_multi.transfer(300, buffer, samples)

        p300_multi.transfer(300, samples, waste)
    
    p300_multi.transfer(150, buffer, samples, blow_out=True, touch_tip=True)
    
    
    return p300_multi


In [10]:
from opentrons import simulate

protocol = simulate.get_protocol_api('2.4')
metadata = {'apiLevel': '2.3'}
x = run(protocol)

C:\Users\bvp22\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\bvp22\.opentrons\robot_settings.json not found. Loading defaults


In [11]:
protocol.commands()

['Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 3 to A4 of NEST 12 Well Reservoir 15 mL on 4',
 'Picking up tip from A1 of Opentrons 96 Filter Tip Rack 200 µL on 2',
 'Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 3 at 1.0 speed',
 'Dispensing 200.0 uL into A4 of NEST 12 Well Reservoir 15 mL on 4 at 1.0 speed',
 'Blowing out at A1 of Opentrons Fixed Trash on 12',
 'Aspirating 200.0 uL from A2 of Corning 96 Well Plate 360 µL Flat on 3 at 1.0 speed',
 'Dispensing 200.0 uL into A4 of NEST 12 Well Reservoir 15 mL on 4 at 1.0 speed',
 'Blowing out at A1 of Opentrons Fixed Trash on 12',
 'Aspirating 200.0 uL from A3 of Corning 96 Well Plate 360 µL Flat on 3 at 1.0 speed',
 'Dispensing 200.0 uL into A4 of NEST 12 Well Reservoir 15 mL on 4 at 1.0 speed',
 'Blowing out at A1 of Opentrons Fixed Trash on 12',
 'Aspirating 200.0 uL from A4 of Corning 96 Well Plate 360 µL Flat on 3 at 1.0 speed',
 'Dispensing 200.0 uL into A4 of NEST 12 Well Reservoir 15 m